In [138]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import xgboost as xgb
import re
import optuna

In [139]:
train_df = pd.read_csv('data/train.csv')
train_df.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [140]:
test_df = pd.read_csv('data/test.csv')
test_df.head(5)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


# EDA

In [141]:
train_df[['Survived', 'Embarked']].groupby('Embarked').mean()

,Survived
Embarked,
C,0.553571
Q,0.389610
S,0.336957


# Feature Engineering

In [142]:
train_df['Sex'] = train_df['Sex'].map({'male' : 0, 'female' : 1})
test_df['Sex'] = test_df['Sex'].map({'male' : 0, 'female' : 1})

In [143]:

train_df['Title'] = train_df['Name'].str.extract(r'(Mr\.|Mrs\.|Ms\.|Miss|Dr\.|Rev\.)')
test_df['Title'] = test_df['Name'].str.extract(r'(Mr\.|Mrs\.|Ms\.|Miss|Dr\.|Rev\.)')

train_df = pd.get_dummies(train_df, columns = ['Title'])
test_df = pd.get_dummies(test_df, columns = ['Title'])


In [144]:
train_df['Age'] = train_df.groupby(['Sex', 'Pclass'])['Age'].transform(lambda x : x.fillna(x.median()))
test_df['Age'] = train_df.groupby(['Sex', 'Pclass'])['Age'].transform(lambda x : x.fillna(x.median()))

In [145]:
train_df['Fare'] = train_df.groupby(['Sex', 'Pclass'])['Fare'].transform(lambda x : x.fillna(x.median()))
test_df['Fare'] = train_df.groupby(['Sex', 'Pclass'])['Fare'].transform(lambda x : x.fillna(x.median()))

In [146]:
train_df = pd.get_dummies(train_df, columns = ['Embarked'])
test_df = pd.get_dummies(test_df, columns = ['Embarked'])

In [147]:
title_one_hot = [col for col in train_df.columns if col.startswith('Title_')]
title_one_hot

['Title_Dr.',
 'Title_Miss',
 'Title_Mr.',
 'Title_Mrs.',
 'Title_Ms.',
 'Title_Rev.']

In [148]:
embarked_one_hot = [col for col in train_df.columns if col.startswith('Embarked_')]
embarked_one_hot

['Embarked_C', 'Embarked_Q', 'Embarked_S']

In [149]:
features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare'] + title_one_hot + embarked_one_hot

In [150]:
df = train_df[['Survived'] + features]
df

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Title_Dr.,Title_Miss,Title_Mr.,Title_Mrs.,Title_Ms.,Title_Rev.,Embarked_C,Embarked_Q,Embarked_S
0,0,3,0,22.0,1,0,7.2500,False,False,True,False,False,False,False,False,True
1,1,1,1,38.0,1,0,71.2833,False,False,False,True,False,False,True,False,False
2,1,3,1,26.0,0,0,7.9250,False,True,False,False,False,False,False,False,True
3,1,1,1,35.0,1,0,53.1000,False,False,False,True,False,False,False,False,True
4,0,3,0,35.0,0,0,8.0500,False,False,True,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,0,27.0,0,0,13.0000,False,False,False,False,False,True,False,False,True
887,1,1,1,19.0,0,0,30.0000,False,True,False,False,False,False,False,False,True
888,0,3,1,21.5,1,2,23.4500,False,True,False,False,False,False,False,False,True
889,1,1,0,26.0,0,0,30.0000,False,False,True,False,False,False,True,False,False


# Training

In [151]:
X = df[features]
y = df['Survived']

In [152]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [153]:
model = RandomForestClassifier(random_state = 0)
model.fit(X_train, y_train)

,n_estimators,100
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [154]:
y_pred = model.predict(X_val)

In [155]:
print('Accuracy: ', accuracy_score(y_val, y_pred))

Accuracy:  0.8379888268156425


In [156]:
test_features = test_df[features]

In [157]:
submission = pd.DataFrame({
    'PassengerId': test_df['PassengerId'],
    'Survived': model.predict(test_features)
})
submission.to_csv('submission.csv', index=False)

In [158]:
model = xgb.XGBClassifier(
    use_label_encoder = False,
    eval_metric='logloss', 
    random_state=42  
)

In [159]:
model.fit(X_train, y_train)

/Users/dominicbankovitch/Documents/Repos/Kaggle/Titanic/venv/lib/python3.13/site-packages/xgboost/training.py:183: UserWarning: [08:56:40] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


,objective,'binary:logistic'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,None
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,'logloss'


In [160]:
y_pred = model.predict(X_val)

In [161]:
accuracy_score(y_val, y_pred)

0.8268156424581006

In [162]:
submission = pd.DataFrame({
    'PassengerId': test_df['PassengerId'],
    'Survived': model.predict(test_features)
})
submission.to_csv('submission_v2_xgb.csv', index=False)